# GAN for Image Generation with CelebA Dataset

This notebook implements a Generative Adversarial Network (GAN) for generating images based on the CelebA dataset. Using PyTorch, we define the GAN structure, load data, and train the model step-by-step.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torchvision.utils import make_grid
import matplotlib.pyplot as plt
import numpy as np

## Step 2: Load and Preprocess the Dataset

In [ ]:
# Set the transformation for the dataset
transform = transforms.Compose([
    transforms.Resize(64),       # Resize to 64x64 pixels
    transforms.CenterCrop(64),
    transforms.ToTensor(),       # Convert to tensor
    transforms.Normalize([0.5], [0.5])  # Normalize to [-1, 1] range for stability
])

# Load the CelebA dataset
data_path = 'path_to_celeba'  # Set this to your CelebA data path
dataset = datasets.ImageFolder(root=data_path, transform=transform)
dataloader = DataLoader(dataset, batch_size=128, shuffle=True)

## Step 3: Define the Generator and Discriminator Models

In [ ]:
# Generator Model
class Generator(nn.Module):
    def __init__(self, nz, ngf, nc):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.ConvTranspose2d(nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            nn.ConvTranspose2d(ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        return self.main(input)

In [ ]:
# Discriminator Model
class Discriminator(nn.Module):
    def __init__(self, nc, ndf):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(nc, ndf, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf, ndf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 2, ndf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 4, ndf * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ndf * 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(ndf * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

## Step 4: Initialize the Models, Loss Function, and Optimizers

In [ ]:
# Hyperparameters
nz = 100  # Size of latent vector
ngf = 64  # Size of feature maps in generator
ndf = 64  # Size of feature maps in discriminator
nc = 3    # Number of channels in the training images (3 for RGB)

# Create the generator and discriminator
netG = Generator(nz, ngf, nc).cuda()
netD = Discriminator(nc, ndf).cuda()

# Loss and optimizer
criterion = nn.BCELoss()
optimizerD = optim.Adam(netD.parameters(), lr=0.0002, betas=(0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=0.0002, betas=(0.5, 0.999))

## Step 5: Training Loop

In [ ]:
num_epochs = 25
fixed_noise = torch.randn(64, nz, 1, 1, device='cuda')

for epoch in range(num_epochs):
    for i, data in enumerate(dataloader, 0):
        # Train Discriminator
        netD.zero_grad()
        real = data[0].cuda()
        batch_size = real.size(0)
        label = torch.full((batch_size,), 1, dtype=torch.float, device='cuda')
        output = netD(real).view(-1)
        lossD_real = criterion(output, label)
        lossD_real.backward()
        D_x = output.mean().item()

        noise = torch.randn(batch_size, nz, 1, 1, device='cuda')
        fake = netG(noise)
        label.fill_(0)
        output = netD(fake.detach()).view(-1)
        lossD_fake = criterion(output, label)
        lossD_fake.backward()
        D_G_z1 = output.mean().item()
        optimizerD.step()

        # Train Generator
        netG.zero_grad()
        label.fill_(1)  # Flip the label for generator's loss
        output = netD(fake).view(-1)
        lossG = criterion(output, label)
        lossG.backward()
        D_G_z2 = output.mean().item()
        optimizerG.step()

        # Print statistics
        if i % 50 == 0:
            print(f'Epoch [{epoch}/{num_epochs}] Batch [{i}/{len(dataloader)}]  Loss D: {lossD_real.item() + lossD_fake.item():.4f}, Loss G: {lossG.item():.4f}, D(x): {D_x:.2f}, D(G(z)): {D_G_z1:.2f}/{D_G_z2:.2f}')

    # Save generated images
    with torch.no_grad():
        fake = netG(fixed_noise).detach().cpu()
    img = make_grid(fake, padding=2, normalize=True)
    plt.figure(figsize=(8, 8))
    plt.imshow(np.transpose(img, (1, 2, 0)))
    plt.axis('off')
    plt.show()

## Step 6: Generate New Images

In [ ]:
# Generate new images after training
with torch.no_grad():
    generated_images = netG(fixed_noise).detach().cpu()
    img = make_grid(generated_images, padding=2, normalize=True)
    plt.figure(figsize=(8, 8))
    plt.imshow(np.transpose(img, (1, 2, 0)))
    plt.axis('off')
    plt.title('Generated Images')
    plt.show()